<a href="https://colab.research.google.com/github/S-ArunaDevi06/AI_generated_review_detection/blob/main/TAM_MAL_CLASSIFICATION_ALBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
#from transformers import TrainingArguments
#from transformers import Trainer

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install --upgrade transformers

In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from tqdm import tqdm

# Load ALBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")  # Replace with desired ALBERT variant
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

# Dataset class
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length, is_test=False):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        text = self.dataframe.iloc[index]["DATA"]
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        if not self.is_test:  # Add label only for training/validation data
            label = self.dataframe.iloc[index]["LABEL"]
            item["LABEL"] = torch.tensor(int(label), dtype=torch.long)
        return item

# Load train and test data
train_df = pd.read_csv("/content/drive/MyDrive/HUMAN_AI_DETECTION/Copy of tam_training_data_hum_ai.csv")
test_df = pd.read_excel("/content/drive/MyDrive/HUMAN_AI_DETECTION/Copy of tam_test_data_hum_ai.xlsx")

# Preprocess labels in training data
for k in range(len(train_df)):
    if train_df["LABEL"][k] == "HUMAN":
        train_df["LABEL"][k] = 0
    else:
        train_df["LABEL"][k] = 1

# Create datasets and dataloaders
train_dataset = TextDataset(train_df, tokenizer, max_length=128)
test_dataset = TextDataset(test_df, tokenizer, max_length=128, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items() if key != "LABEL"}
        labels = batch["LABEL"].to(device)

        # Forward pass
        outputs = model(**inputs, labels=labels)

        # Backward pass and optimization
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

# Prediction on test data
model.eval()
test_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
        test_predictions.extend(predictions.cpu().numpy())

# Add predictions to test dataframe
test_df["PREDICTED_LABEL"] = test_predictions
test_df["PREDICTED_LABEL"] = test_df["PREDICTED_LABEL"].map({0: "human", 1: "ai"})

# Save predictions to a file
test_df.to_csv("/content/drive/MyDrive/HUMAN_AI_DETECTION/test_predictions_albert.tsv", index=False)

print("Predictions saved to test_predictions_albert.csv")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-2-256b2d95593a>:47: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the ca

KeyboardInterrupt: 

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
from tqdm import tqdm

# Load ALBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")  # Replace with desired ALBERT variant
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

# Dataset class
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length, is_test=False):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        text = self.dataframe.iloc[index]["DATA"]
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        if not self.is_test:  # Add label only for training/validation data
            label = self.dataframe.iloc[index]["LABEL"]
            item["LABEL"] = torch.tensor(int(label), dtype=torch.long)
        return item

# Load train and test data
train_df = pd.read_csv("/content/drive/MyDrive/HUMAN_AI_DETECTION/Copy of tam_training_data_hum_ai.csv")
test_df = pd.read_excel("/content/drive/MyDrive/HUMAN_AI_DETECTION/Copy of tam_test_data_hum_ai.xlsx")

# Preprocess labels in training data
train_df["LABEL"] = train_df["LABEL"].map({"HUMAN": 0, "AI": 1})

# Create datasets and dataloaders
train_dataset = TextDataset(train_df, tokenizer, max_length=128)
test_dataset = TextDataset(test_df, tokenizer, max_length=128, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items() if key != "LABEL"}
        labels = batch["LABEL"].to(device)

        # Forward pass
        outputs = model(**inputs, labels=labels)

        # Backward pass and optimization
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

# Prediction on test data
model.eval()
test_predictions = []
true_labels = []  # Store ground truth labels

with torch.no_grad():
    for batch in tqdm(test_loader):
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)

        test_predictions.extend(predictions.cpu().numpy())

# Convert test predictions to labels
test_df["PREDICTED_LABEL"] = test_predictions
test_df["PREDICTED_LABEL"] = test_df["PREDICTED_LABEL"].map({0: "HUMAN", 1: "AI"})

# Extract ground truth labels from test data (if available)
if "LABEL" in test_df.columns:
    test_df["LABEL"] = test_df["LABEL"].map({"HUMAN": 0, "AI": 1})  # Ensure numeric labels
    true_labels = test_df["LABEL"].tolist()

    # Compute accuracy and macro F1-score
    accuracy = accuracy_score(true_labels, test_predictions)
    macro_f1 = f1_score(true_labels, test_predictions, average="macro")

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Macro F1 Score: {macro_f1:.4f}")

# Save predictions to a file
test_df.to_csv("/content/drive/MyDrive/HUMAN_AI_DETECTION/test_predictions_albert.tsv", index=False)

print("Predictions saved to test_predictions_albert.tsv")


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 51/51 [19:00<00:00, 22.37s/it]


Epoch 1/3, Loss: 0.2247188677828686


100%|██████████| 51/51 [18:50<00:00, 22.17s/it]


Epoch 2/3, Loss: 0.17265371178441188


100%|██████████| 51/51 [18:26<00:00, 21.71s/it]


Epoch 3/3, Loss: 0.17175445508430986


100%|██████████| 7/7 [00:47<00:00,  6.74s/it]

Predictions saved to test_predictions_albert.tsv


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
from tqdm import tqdm

# Load ALBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

# Dataset class
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        text = self.dataframe.iloc[index]["DATA"]
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        label = self.dataframe.iloc[index]["LABEL"]
        item["LABEL"] = torch.tensor(int(label), dtype=torch.long)
        return item

# Load and preprocess training data
train_df = pd.read_csv("/content/drive/MyDrive/HUMAN_AI_DETECTION/Copy of tam_training_data_hum_ai.csv")

# Convert labels to numeric
train_df["LABEL"] = train_df["LABEL"].map({"HUMAN": 0, "AI": 1})

# Create dataset object
full_dataset = TextDataset(train_df, tokenizer, max_length=128)

# Split dataset into 80% training and 20% validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items() if key != "LABEL"}
        labels = batch["LABEL"].to(device)

        # Forward pass
        outputs = model(**inputs, labels=labels)

        # Backward pass and optimization
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

# Evaluation on validation set
model.eval()
val_predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(val_loader):
        inputs = {key: val.to(device) for key, val in batch.items() if key != "LABEL"}
        labels = batch["LABEL"].to(device)

        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)

        val_predictions.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy and macro F1-score
accuracy = accuracy_score(true_labels, val_predictions)
macro_f1 = f1_score(true_labels, val_predictions, average="macro")

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Macro F1 Score: {macro_f1:.4f}")


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 41/41 [14:59<00:00, 21.94s/it]


Epoch 1/3, Loss: 0.2590751827399178


100%|██████████| 41/41 [14:53<00:00, 21.78s/it]


Epoch 2/3, Loss: 0.17632924838037026


100%|██████████| 41/41 [14:53<00:00, 21.80s/it]


Epoch 3/3, Loss: 0.16990276753175548


100%|██████████| 11/11 [01:15<00:00,  6.88s/it]

Validation Accuracy: 0.9136
Validation Macro F1 Score: 0.9122


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from tqdm import tqdm

# Load ALBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")  # Replace with desired ALBERT variant
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

# Dataset class
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length, is_test=False):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        text = self.dataframe.iloc[index]["DATA"]
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        if not self.is_test:  # Add label only for training/validation data
            label = self.dataframe.iloc[index]["LABEL"]
            item["LABEL"] = torch.tensor(int(label), dtype=torch.long)
        return item

# Load train and test data
train_df = pd.read_csv("/content/drive/MyDrive/HUMAN_AI_DETECTION/Copy of mal_training_data_hum_ai.csv")
test_df = pd.read_excel("/content/drive/MyDrive/HUMAN_AI_DETECTION/Copy of mal_test_data_hum_ai.xlsx")

# Preprocess labels in training data
for k in range(len(train_df)):
    if train_df["LABEL"][k] == "HUMAN":
        train_df["LABEL"][k] = 0
    else:
        train_df["LABEL"][k] = 1

# Create datasets and dataloaders
train_dataset = TextDataset(train_df, tokenizer, max_length=128)
test_dataset = TextDataset(test_df, tokenizer, max_length=128, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items() if key != "LABEL"}
        labels = batch["LABEL"].to(device)

        # Forward pass
        outputs = model(**inputs, labels=labels)

        # Backward pass and optimization
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

# Prediction on test data
model.eval()
test_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
        test_predictions.extend(predictions.cpu().numpy())

# Add predictions to test dataframe
test_df["PREDICTED_LABEL"] = test_predictions
test_df["PREDICTED_LABEL"] = test_df["PREDICTED_LABEL"].map({0: "human", 1: "ai"})

# Save predictions to a file
test_df.to_csv("/content/drive/MyDrive/HUMAN_AI_DETECTION/test_predictions_albert_mal.tsv", index=False)

print("Predictions saved to test_predictions_albert_mal.csv")



Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-7-aee7d22cef46>:45: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the ca

Epoch 1/3, Loss: 0.3473792771995068


100%|██████████| 50/50 [18:23<00:00, 22.08s/it]


Epoch 2/3, Loss: 0.19137996390461923


100%|██████████| 50/50 [18:25<00:00, 22.11s/it]


Epoch 3/3, Loss: 0.17483973436057568


100%|██████████| 13/13 [01:33<00:00,  7.22s/it]

Predictions saved to test_predictions_albert_mal.csv


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
from tqdm import tqdm

# Load ALBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)

# Dataset class
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        text = self.dataframe.iloc[index]["DATA"]
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        label = self.dataframe.iloc[index]["LABEL"]
        item["LABEL"] = torch.tensor(int(label), dtype=torch.long)
        return item

# Load and preprocess training data
train_df = pd.read_csv("/content/drive/MyDrive/HUMAN_AI_DETECTION/Copy of mal_training_data_hum_ai.csv")

# Convert labels to numeric
train_df["LABEL"] = train_df["LABEL"].map({"HUMAN": 0, "AI": 1})

# Create dataset object
full_dataset = TextDataset(train_df, tokenizer, max_length=128)

# Split dataset into 80% training and 20% validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items() if key != "LABEL"}
        labels = batch["LABEL"].to(device)

        # Forward pass
        outputs = model(**inputs, labels=labels)

        # Backward pass and optimization
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

# Evaluation on validation set
model.eval()
val_predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(val_loader):
        inputs = {key: val.to(device) for key, val in batch.items() if key != "LABEL"}
        labels = batch["LABEL"].to(device)

        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)

        val_predictions.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute accuracy and macro F1-score
accuracy = accuracy_score(true_labels, val_predictions)
macro_f1 = f1_score(true_labels, val_predictions, average="macro")

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Macro F1 Score: {macro_f1:.4f}")


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 40/40 [14:49<00:00, 22.23s/it]


Epoch 1/3, Loss: 0.33393300995230674


100%|██████████| 40/40 [14:38<00:00, 21.97s/it]


Epoch 2/3, Loss: 0.20815059645101427


100%|██████████| 40/40 [14:46<00:00, 22.16s/it]


Epoch 3/3, Loss: 0.1987454836256802


100%|██████████| 10/10 [01:15<00:00,  7.59s/it]

Validation Accuracy: 0.9500
Validation Macro F1 Score: 0.9499
